In [8]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
m=n
q=floor(Int64,n/m)
R=ones(Float64,m)./n
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=0#ceil(Int64, n/7)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=5
Number of equality constraints: l=0


In [9]:
using ctpPOP2

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP2.get_info(x,f,g,h,sparse=false);

In [10]:
k=2
println("Relaxed order: k=",k)

Relaxed order: k=2


In [11]:
using ctpPOP2

opt_val1=ctpPOP2.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=6
  Size of the largest block: s^max=21
  Number of equality trace constraints: zeta=211
Modeling time:
  0.008223 seconds (15.08 k allocations: 1.646 MiB)
iter=1   val=-3.686168087218646   gap=0.00027657864007725827   feas=3.6776719963184967
iter=2   val=-0.7336375057818572   gap=0.6856325343440921   feas=0.8991692337074143
iter=4   val=-0.12492088122077133   gap=0.5480682160558514   feas=0.9663385579604783
iter=8   val=-0.20162686592420126   gap=0.29493879408952245   feas=0.6031956649840721
iter=16   val=-0.2342183284881777   gap=0.4197567815939036   feas=0.1948203344504864
iter=32   val=-0.29443464463785857   gap=0.049828780789080054   feas=0.1361713815862133
iter=64   val=-0.24517263578816328   gap=0.16324400746297926   feas=0.08820866001129647
iter=128   val=-0.2796152437646705   gap=0.01771541931427684   feas=0.02795979947540347
iter=256   val=-0.3119483433113311   gap=0.01009023525786

-0.3240847068919046

In [12]:
using ctpPOP2

opt_val2=ctpPOP2.POP_dense_SOS(x,f,g,h,k,tol=1e-3)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 126             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 6               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

-0.32471345686928904

In [14]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)